# ML Notebook

In [117]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
import xgboost as xgb

In [118]:
df = pd.read_csv('db/vw/golf.csv')
df = df.dropna(subset=['year', 'range', 'location', 'price'])

In [119]:
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

In [120]:
df_cleaned = df.drop_duplicates()


In [121]:
mean_price = df_cleaned['price'].mean()
std_price = df_cleaned['price'].std()

df_cleaned = df_cleaned[(df_cleaned['price'] <= mean_price + 2 * std_price) & (df_cleaned['price'] >= mean_price - 2 * std_price)]

In [122]:
f1_unique = df['f1'].unique()
f2_unique = df['f2'].unique()
f3_unique = df['f3'].unique()

df_cleaned.loc[df['f2'].isin(f3_unique), 'f2'] = np.nan
df_cleaned.loc[df['f1'].isin(f3_unique), 'f1'] = np.nan
df_cleaned.loc[df['f1'].isin(f2_unique), 'f1'] = np.nan

rare_f1 = df_cleaned['f1'].value_counts()[df_cleaned['f1'].value_counts() < 10].index
df_cleaned.loc[df_cleaned['f1'].isin(rare_f1), 'f1'] = np.nan

rare_f2 = df_cleaned['f2'].value_counts()[df_cleaned['f2'].value_counts() < 10].index
df_cleaned.loc[df_cleaned['f2'].isin(rare_f2), 'f2'] = np.nan

rare_f3 = df_cleaned['f3'].value_counts()[df_cleaned['f3'].value_counts() < 10].index
df_cleaned.loc[df_cleaned['f3'].isin(rare_f3), 'f3'] = np.nan



df_cleaned['f2'].value_counts()

1.4 мт (75 к.с.)                     360
1.6 tdi mt (105 к.с.)                171
2.0 tdi dsg (150 к.с.)               109
1.6 мт (105 к.с.)                    104
1.4 tsi mt (122 к.с.)                 93
1.6 мт (102 к.с.)                     91
1.4 мт (80 к.с.)                      86
1.6 ат (102 к.с.)                     78
1.6 мт (101 к.с.)                     73
1.6 tdi mt (110 к.с.)                 64
1.6 tdi bluemotion mt (105 к.с.)      59
1.6 mt (102 к.с.)                     49
1.9 tdi mt (105 к.с.)                 46
2.0 tdi mt (150 к.с.)                 44
1.6 fsi мт (115 к.с.)                 38
1.6 tdi dsg (105 к.с.)                38
1.6 tdi dsg (110 к.с.)                38
1.6 fsi мт (110 к.с.)                 36
1.4 tsi dsg (122 к.с.)                35
2.0 tdi mt (140 к.с.)                 34
1.6 ат (101 к.с.)                     30
1.6 tdi mt (115 к.с.)                 29
1.9 tdi мт (90 к.с.)                  28
1.4 tsi mt (140 к.с.)                 26
1.9 tdi мт (105 

In [123]:

df_cleaned['date'] = pd.to_datetime(df_cleaned['date'])
df_cleaned['year'] = df_cleaned['year'].astype(int)
df_cleaned['hp'] = df_cleaned['hp'].astype(int)
df_cleaned['range'] = df_cleaned['range'].astype(int)
df_cleaned['kp'] = df_cleaned['kp'].astype(int)
df_cleaned['dtp'] = df_cleaned['dtp'].astype(int)
df_cleaned['subtitle'] = df_cleaned['subtitle'].astype(str)

In [124]:
df_cleaned = df_cleaned[df_cleaned['year'] >= 2004]
df_cleaned.loc[df_cleaned['kp'] == -1, 'kp'] = 0
df_cleaned.loc[df_cleaned['hp'] == -1, 'hp'] = df_cleaned['hp'].mean()

In [125]:
def split_location(location):
  if ',' in location:
    location = location.split(',')[0].strip()
  parts = location.split('•')
  return parts[1].strip() if len(parts) > 1 else location.strip()

df_cleaned['location'] = df_cleaned['location'].apply(split_location)

In [126]:
df_cleaned['location'].unique()

array(['Вінниця', 'Вінницька обл.', 'Львівська обл.', 'Львів',
       'Черкаська обл.', 'Миколаїв', 'Одеса', 'Житомир',
       'Івано-Франківськ', 'Закарпатська обл.', 'Івано-Франківська обл.',
       'Київ', 'Полтава', 'Тернопільська обл.', 'Одеська обл.',
       "Кам'янець-Подільський", 'Київська обл.', 'Хмельницька обл.',
       'Житомирська обл.', 'Ужгород', 'Миколаївська обл.', 'Черкаси',
       'Тернопіль', 'Хмельницький', 'Запоріжжя',
       'Дніпро (Дніпропетровськ)', 'Рівне', 'Дніпропетровська обл.',
       'Харків', 'Чернігівська обл.', 'Суми', 'Волинська обл.',
       'Чернівці', 'Харківська обл.', 'Кіровоградська обл.',
       'Кропивницький (Кіровоград)', 'Сумська обл.', 'Луцьк',
       'Полтавська обл.', 'Чернівецька обл.', 'Чернігів', 'Бердичів',
       'Рівненська обл.', 'Мукачево', 'Червоноград', 'Донецька обл.',
       'Запорізька обл.', 'Кривий Ріг', 'Коломия', 'Херсонська обл.',
       'Херсон', 'Солонка', 'Тячів', 'Ніжин'], dtype=object)

In [127]:
def split_subtitle(subtitle):
  return subtitle.split('•')[0]

df_cleaned['new_subtitle1'] = df_cleaned['subtitle'].apply(split_subtitle)
df_cleaned['new_subtitle1'] = df_cleaned['new_subtitle1'].str.strip()

df_cleaned = df_cleaned[df_cleaned['new_subtitle1'].map(df_cleaned['new_subtitle1'].value_counts()) >= 10]

# Encode the new_subtitle column
label_encoder = LabelEncoder()
df_cleaned['new_subtitle1_encoded'] = label_encoder.fit_transform(df_cleaned['new_subtitle1'])
df_cleaned['location_encoded'] = label_encoder.fit_transform(df_cleaned['location'])
df_cleaned['f1_encoded'] = label_encoder.fit_transform(df_cleaned['f1'])
df_cleaned['f2_encoded'] = label_encoder.fit_transform(df_cleaned['f2'])
df_cleaned['f3_encoded'] = label_encoder.fit_transform(df_cleaned['f3'])


In [128]:
def split_subtitle(subtitle):
    parts = subtitle.split('•')
    return parts[1] if len(parts) > 1 else ''

df_cleaned['new_subtitle2'] = df_cleaned['subtitle'].apply(split_subtitle)
df_cleaned['new_subtitle2'] = df_cleaned['new_subtitle2'].str.strip()


# Encode the new_subtitle column
df_cleaned['new_subtitle2_encoded'] = label_encoder.fit_transform(df_cleaned['new_subtitle2'])


In [129]:
df_cleaned['mileage_per_year'] = df_cleaned['range'] / (2025 - df_cleaned['year'])

In [130]:
df_cleaned

,link,date,price,title,subtitle,f1,f2,f3,year,hp,...,desc,new_subtitle1,new_subtitle1_encoded,location_encoded,f1_encoded,f2_encoded,f3_encoded,new_subtitle2,new_subtitle2_encoded,mileage_per_year
0,https://auto.ria.com/uk/auto_volkswagen_golf_3...,2024-12-15,8500.0,volkswagen golf 2011,vi покоління/typ 5k • 1.6 tdi dsg (105 к.с.),vi покоління/typ 5k,1.6 tdi dsg (105 к.с.),NaN,2011,105.000000,...,продаю свою машину у гарному стані . гаражне з...,vi покоління/typ 5k,3,5,2,16,5,1.6 tdi dsg (105 к.с.),45,13.357143
3,https://auto.ria.com/uk/auto_volkswagen_golf_3...,2024-12-15,9400.0,volkswagen golf 2011,nan,NaN,NaN,NaN,2011,56.093478,...,"golf 6 комплектація r-line ,1.4 tsi 162 к.с 11...",nan,1,5,6,40,5,,0,14.785714
4,https://auto.ria.com/uk/auto_volkswagen_golf_3...,2024-12-15,6300.0,volkswagen golf 2007,v покоління/typ 1k,v покоління/typ 1k,NaN,NaN,2007,56.093478,...,"гарний стан, по мотору ходовій питань нема, ку...",v покоління/typ 1k,2,4,1,40,5,,0,13.277778
5,https://auto.ria.com/uk/auto_volkswagen_golf_3...,2024-12-15,12500.0,volkswagen golf 2018,vii покоління (fl)/typ 5g • 1.6 tdi mt (115 к....,vii покоління (fl)/typ 5g,1.6 tdi mt (115 к.с.),comfortline,2018,115.000000,...,"продається vw golf 2018 року, обслужений, цент...",vii покоління (fl)/typ 5g,4,23,3,24,1,1.6 tdi mt (115 к.с.),54,42.857143
6,https://auto.ria.com/uk/auto_volkswagen_golf_3...,2024-12-15,8800.0,volkswagen golf 2016,nan,NaN,NaN,NaN,2016,56.093478,...,"авто з 2021 року в одній сім?ї , приганялось з...",nan,1,22,6,40,5,,0,32.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5067,https://auto.ria.com/uk/auto_volkswagen_golf_3...,2024-12-15,8600.0,volkswagen golf 2012,vi покоління/typ 5k • 1.6 tdi mt (105 к.с.),vi покоління/typ 5k,1.6 tdi mt (105 к.с.),NaN,2012,105.000000,...,"продам економний, комфортний сімейний автомобі...",vi покоління/typ 5k,3,35,2,20,5,1.6 tdi mt (105 к.с.),50,13.000000
5070,https://auto.ria.com/uk/auto_volkswagen_golf_3...,2024-12-15,6999.0,volkswagen golf 2010,vi покоління/typ 5k • 1.4 tsi mt (122 к.с.) • ...,vi покоління/typ 5k,1.4 tsi mt (122 к.с.),comfortline,2010,122.000000,...,автомобіль в нормальному стані вкладень не пот...,vi покоління/typ 5k,3,45,2,5,1,1.4 tsi mt (122 к.с.),24,14.000000
5076,https://auto.ria.com/uk/auto_volkswagen_golf_3...,2024-12-15,8100.0,volkswagen golf 2012,vi покоління/typ 5k,vi покоління/typ 5k,NaN,NaN,2012,56.093478,...,"продаю власний автомобіль, комплектація style,...",vi покоління/typ 5k,3,33,2,40,5,,0,21.461538
5081,https://auto.ria.com/uk/auto_volkswagen_golf_3...,2024-12-15,5100.0,volkswagen golf 2007,v покоління/typ 1k • 1.4 tsi dsg (170 к.с.) • ...,v покоління/typ 1k,NaN,base,2007,170.000000,...,авто в технічно гарному стані все працює гарно...,v покоління/typ 1k,2,11,1,40,0,1.4 tsi dsg (170 к.с.),23,16.111111


In [142]:
X = df_cleaned[['year', 'range', 'kp', 'hp', 'dtp','fuel','f1_encoded','f2_encoded','f3_encoded', 'mileage_per_year', 'location_encoded']]
y = df_cleaned['price']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Create and train the model
model = RandomForestRegressor(n_estimators=500, max_depth=10)
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = model.score(X_test, y_test)
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
print(f'Mean Squared Error: {mse}')
print(f'Root Mean Squared Error: {rmse}')
print(f'Mean Absolute Percentage Error: {mape}')
print(f'R^2: {r2}')

Mean Squared Error: 1765915.2663568275
Root Mean Squared Error: 1328.87744595084
Mean Absolute Percentage Error: 10.05565473954573
R^2: 0.8565762070308749


In [143]:
model.feature_importances_

array([0.85958532, 0.03638089, 0.00840935, 0.02425001, 0.00432972,
       0.00535466, 0.01302177, 0.00440187, 0.00389819, 0.02272359,
       0.01764463])

In [133]:


# Perform cross-validation
cv_scores = cross_val_score(model, X, y, cv=5, scoring='neg_mean_squared_error')

# Convert negative MSE to positive
cv_scores = -cv_scores

# Calculate mean and standard deviation of the cross-validation scores
mean_cv_score = np.mean(cv_scores)
std_cv_score = np.std(cv_scores)

print(f'Cross-Validation Mean Squared Error: {mean_cv_score}')
print(f'Root Cross-Validation Mean Squared Error: {np.sqrt(mean_cv_score)}')

Cross-Validation Mean Squared Error: 1790565.7016096853
Root Cross-Validation Mean Squared Error: 1338.1202119427408
